<a href="https://colab.research.google.com/github/dhanushnayak/Code_generator/blob/main/falcoder_imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun 21 23:35:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U  git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [4]:
model_id = "mrm8488/falcon-7b-ft-codeAlpaca_20k-v2"

config =  PeftConfig.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,return_dict=True,load_in_8bit=True, device_map={"":0}, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
model = PeftModel.from_pretrained(model,model_id)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Paramete

In [7]:
def generate(instruction,
             max_new_tokens=128,
             temperature=0.1,
             top_p=0.75,
             top_k=40,
             num_beams=4,
             **kwargs
             ):
  prompt = instruction + '\n ### Solution:\n'
  print(prompt)
  inputs = tokenizer(prompt, return_tensors='pt')
  print("INPUT -> ",inputs.keys())
  input_ids = inputs['input_ids'].to("cuda")
  attention_mask = inputs['attention_mask'].to("cuda")
  generation_config = GenerationConfig(
      temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
  with torch.no_grad():
    gen_output = model.generate(
        input_ids = input_ids,
        attention_mask = attention_mask,
        generation_config =  generation_config,
        return_dict_in_generate=True,
        output_scores = True,
        max_new_tokens =  max_new_tokens,
        early_stopping = True
    )
    print(gen_output)
    s = gen_output.sequences[0]
    output = tokenizer.decode(s)
    return output




In [8]:
instruction = "Design a class for representing a person in Python."

In [9]:
solution = generate(instruction)
print(solution)

Design a class for representing a person in Python.
 ### Solution:

INPUT ->  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
BeamSearchDecoderOnlyOutput(sequences=tensor([[14188,   241,  1234,   312, 12822,   241,   923,   272, 16621,    25,
          1212, 19468, 19157,    37,   193,  3423,  5639,    37,   742,   984,
           204,   999,  7618, 17888,  1047,    23,  1536,    23,  2521,    23,
          7952,  2045,   854,  1557,    25,  2827,   204,    40,  1536,   854,
          1557,    25,   484,   204,    40,  2521,   854,  1557,    25, 21054,
           204,    40,  7952,  2273,   984,   577,    74,  2827,    19,  1047,
          2045,   854,  1161,  1557,    25,  2827,  2273,   984,   577,    74,
           484,    19,  1047,  2045,   854,  1161,  1557,    25,   484,  2273,
           984,   577,    74, 21054,    19,  1047,  2045,   854,  1161,  1557,
            25, 21054,  2273,   984,   889,    74,  2827,    19,  1047,    23,
          1536,  2045,   854,  15